# **Setup Part**

## Install Libraries

In [ ]:
!pip install parsivar
!pip install emoji
!pip install -U tensorflow
!pip install fastapi kaleido python-multipart uvicorn
!pip install dadmatools
!pip install -U accelerate
!pip install -U transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.3 MB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 22.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8


## Import Necessary Libraries

In [ ]:
# fix the problem with numpy version's attributes and warnings
import numpy as np

def dummy_npwarn_decorator_factory():
  def npwarn_decorator(x):
    return x
  return npwarn_decorator

np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)


import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch.nn as nn
from transformers import AutoTokenizer, get_linear_schedule_with_warmup, AdamW, get_scheduler,\
 AutoModel, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding,\
 pipeline
from datasets import DatasetDict, Dataset, load_dataset
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from tqdm.notebook import tqdm
import pandas as pd
import os
from google.colab import drive

## Preprocessing Tools

In [ ]:
from parsivar import Normalizer
import regex as re

In [ ]:
def clean_persian_text(text):
    # Step 0: Use Parsivar to normalize data
    normalizer = Normalizer()
    text = normalizer.normalize(text)

    text = text.replace('٥', '5')

    # Step 1: Remove any English character from the text
    text = re.sub(r'[a-zA-Z]', '', text)

    # Step 2: Remove letters repeated more than twice
    text = re.sub(r'(.)\1{2,}', r'\1', text)

    # Step 3: Remove any Arabic diacritics from the text
    arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(arabic_diacritics, '', text)

    # Step 4: Remove the hashtag sign (“#”) from the text while keeping the information included in the hashtags
    text = re.sub(r'#', ' ', text)
    text = text.replace('_', ' ')

    # Step 5: Remove Persian and Arabic numeric characters from the text
    text = re.sub(r'\d+', '', text)

    # Step 6: Last normalization
    text = normalizer.normalize(text)

    return text

# **Prediction Part**

## Load the Model

In [ ]:
# Setting checkpoint
checkpoint = 'xlm-roberta-large'

In [ ]:
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

label_dict_reverse = {0: 'SAD', 1: 'HAPPY', 2: 'SURPRISE', 3: 'HATE', 4: 'FEAR', 5: 'ANGRY', 6: 'OTHER'}

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("/content/gdrive/MyDrive/DL_model_v0.3")

## Prediction Function

In [ ]:
def predict(model, tokenizer, tweets, label_dict_reverse):
    cleaned_tweets = []
    for tweet in tweets:
        cleaned_tweets.append(clean_persian_text(tweet))

    nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

    # Use the pipeline to predict the sentiment of a sentence
    results = nlp(cleaned_tweets)
    for result, tweet in zip(results, tweets):
        result['label'] = label_dict_reverse[int(result['label'].lstrip('LABEL_'))]
        result['tweet'] = tweet

    # The result is a list of dictionaries. Each dictionary contains the
    # `label` and `score` for a sentence.
    return results

## Read Tweets from CSV

In [ ]:
# Path to the CSV file
# file_path = 'test_tweets.csv'
file_path = 'test_tweets.csv'

# Read the CSV file into a pandas DataFrame
tweets = pd.read_csv(file_path)
tweets

,tweets
0,واقعا حال به هم زنه این حجم از داستان سرایی در...
1,کتاب امروز به دستم رسید. جنس برگه هاش خوب بودن...
2,انصافا چقدر فک کردی اینو نوشتی خخخ! ولی احسنت ...
3,مربی خارجی حق ندارد ، مربی خارجی بی‌قد و قواره...


## Predict Tweets

## List Tweet

In [ ]:
result = predict(model, tokenizer, tweets['tweets'], label_dict_reverse)
result

[{'label': 'SAD',
  'score': 0.782342255115509,
  'tweet': 'واقعا حال به هم زنه این حجم از داستان سرایی درباره تجاوز یا چیز\u200cهاي شبیه به اون برا جذب لایک و توجه'},
 {'label': 'HAPPY',
  'score': 0.7656766772270203,
  'tweet': 'کتاب امروز به دستم رسید. جنس برگه هاش خوب بودن. رنگهای شادی داشت. اسم شخصیتهای داستانها خنده دار و جالب بود. داستانهاش هم تکراری نبودن. فقط چون کل سی جلد یکجا جمع شده کمی دست رو خسته میکنه'},
 {'label': 'HAPPY',
  'score': 0.476645290851593,
  'tweet': 'انصافا چقدر فک کردی اینو نوشتی خخخ! ولی احسنت جالب بود'},
 {'label': 'ANGRY',
  'score': 0.8665198683738708,
  'tweet': 'مربی خارجی حق ندارد ، مربی خارجی بی\u200cقد و قواره و . ولی میدانید که مردم هم حق اعتراض ندارن ؟ مردم هم خس و خاشاکن ؟ مردم هم بد بختن و شما از همه بهتر میبنید و میدانید ! تریبون اصلی مملکت هم در اختیار شماس ، چه قدمی برداشتید جز سانسور و جز شانتاژ رسانه\u200cای ؟'}]

## Single Tweet

In [ ]:
tweet = 'tweet goes here'
result = predict(model, tokenizer, [tweet], label_dict_reverse)
result